In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('vsf20-21data.txt')
winter2 = pd.read_csv ('vsf21-22data.txt')
winter3 = pd.read_csv ('vsf22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
8,VSF,2021-12-01 06:54:00,T,-SN
9,VSF,2021-12-01 07:54:00,T,-SN BR
35,VSF,2021-12-02 08:23:00,T,-SN
36,VSF,2021-12-02 08:46:00,T,-SN BR
37,VSF,2021-12-02 08:54:00,T,-SN BR
...,...,...,...,...
3162,VSF,2022-02-27 23:13:00,T,-SN
3163,VSF,2022-02-27 23:20:00,T,SN
3164,VSF,2022-02-27 23:33:00,T,-SN
3165,VSF,2022-02-27 23:49:00,T,-SN


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 54]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 54]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 54]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
vsf = LocationInfo(
    'Springfield', 'Vermont', 
    latitude=43.2987,
    longitude=-72.4820, 
    timezone='America/Vermont'
)

vsf

LocationInfo(name='Springfield', region='Vermont', timezone='America/Vermont', latitude=43.2987, longitude=-72.482)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(vsf.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(vsf.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(vsf.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:03:12  2020-12-01 16:14:57  2020-12-01
1   2020-12-02 07:04:16  2020-12-02 16:14:40  2020-12-02
2   2020-12-03 07:05:19  2020-12-03 16:14:25  2020-12-03
3   2020-12-04 07:06:21  2020-12-04 16:14:13  2020-12-04
4   2020-12-05 07:07:21  2020-12-05 16:14:04  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:34:05  2021-02-24 17:32:41  2021-02-24
86  2021-02-25 06:32:29  2021-02-25 17:33:58  2021-02-25
87  2021-02-26 06:30:52  2021-02-26 17:35:15  2021-02-26
88  2021-02-27 06:29:14  2021-02-27 17:36:31  2021-02-27
89  2021-02-28 06:27:35  2021-02-28 17:37:47  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:02:56  2021-12-01 16:15:01  2021-12-01
1   2021-12-02 07:04:01  2021-12-02 16:14:43  2021-12-02
2   2021-12-03 07:05:04  2021-12-03 16:14:28  2021-12-03
3   2021-12-04 07:06:06  2021-12-04 16:14:16  2021-12-04
4   2021-12-05 07:07:07  2021-12-05 16:14:06  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:34:28  2022-02-24 17:32:22  2022-02-24
86  2022-02-25 06:32:52  2022-02-25 17:33:39  2022-02-25
87  2022-02-26 06:31:15  2022-02-26 17:34:56  2022-02-26
88  2022-02-27 06:29:38  2022-02-27 17:36:13  2022-02-27
89  2022-02-28 06:27:59  2022-02-28 17:37:29  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:02:40  2022-12-01 16:15:05  2022-12-01
1   2022-12-02 07:03:45  2022-12-02 16:14:47  2022-12-02
2   2022-12-03 07:04:49  2022-12-03 16:14:32  2022-12-03
3   2022-12-04 07:05:51  2022-12-04 16:14:18  2022-12-04
4   2022-12-05 07:06:52  2022-12-05 16:14:08  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:34:51  2023-02-24 17:32:03  2023-02-24
86  2023-02-25 06:33:15  2023-02-25 17:33:20  2023-02-25
87  2023-02-26 06:31:39  2023-02-26 17:34:37  2023-02-26
88  2023-02-27 06:30:01  2023-02-27 17:35:54  2023-02-27
89  2023-02-28 06:28:23  2023-02-28 17:37:10  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,VSF,2020-12-02 20:54:00,0.0001,-SN,2020-12-02,2020-12-02 07:04:16,2020-12-02 16:14:40
1,VSF,2020-12-03 01:54:00,0.0001,-SN,2020-12-03,2020-12-03 07:05:19,2020-12-03 16:14:25
2,VSF,2020-12-05 22:54:00,0.2500,-SN,2020-12-05,2020-12-05 07:07:21,2020-12-05 16:14:04
3,VSF,2020-12-05 23:54:00,0.2500,-SN,2020-12-05,2020-12-05 07:07:21,2020-12-05 16:14:04
4,VSF,2020-12-06 00:54:00,0.0001,-SN,2020-12-06,2020-12-06 07:08:20,2020-12-06 16:13:57
...,...,...,...,...,...,...,...
157,VSF,2021-02-27 10:54:00,0.0001,-SN,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31
158,VSF,2021-02-27 11:54:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31
159,VSF,2021-02-27 12:54:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31
160,VSF,2021-02-27 13:54:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,VSF,2021-12-01 06:54:00,0.0001,-SN,2021-12-01,2021-12-01 07:02:56,2021-12-01 16:15:01
1,VSF,2021-12-01 07:54:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:02:56,2021-12-01 16:15:01
2,VSF,2021-12-02 08:54:00,0.0001,-SN BR,2021-12-02,2021-12-02 07:04:01,2021-12-02 16:14:43
3,VSF,2021-12-02 09:54:00,0.0001,-SN BR,2021-12-02,2021-12-02 07:04:01,2021-12-02 16:14:43
4,VSF,2021-12-02 10:54:00,0.0001,-SN BR,2021-12-02,2021-12-02 07:04:01,2021-12-02 16:14:43
...,...,...,...,...,...,...,...
160,VSF,2022-02-25 22:54:00,0.0001,-SN,2022-02-25,2022-02-25 06:32:52,2022-02-25 17:33:39
161,VSF,2022-02-25 23:54:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:32:52,2022-02-25 17:33:39
162,VSF,2022-02-26 00:54:00,0.0001,-SN,2022-02-26,2022-02-26 06:31:15,2022-02-26 17:34:56
163,VSF,2022-02-27 22:54:00,0.0001,-SN,2022-02-27,2022-02-27 06:29:38,2022-02-27 17:36:13


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,VSF,2022-12-11 19:54:00,0.0001,-SN,2022-12-11,2022-12-11 07:12:27,2022-12-11 16:13:56
1,VSF,2022-12-11 20:54:00,0.7600,-SN BR,2022-12-11,2022-12-11 07:12:27,2022-12-11 16:13:56
2,VSF,2022-12-11 21:54:00,0.0001,-SN,2022-12-11,2022-12-11 07:12:27,2022-12-11 16:13:56
3,VSF,2022-12-11 22:54:00,0.2500,-SN BR,2022-12-11,2022-12-11 07:12:27,2022-12-11 16:13:56
4,VSF,2022-12-11 23:54:00,0.5100,-SN BR,2022-12-11,2022-12-11 07:12:27,2022-12-11 16:13:56
...,...,...,...,...,...,...,...
174,VSF,2023-02-27 03:54:00,0.2500,-SN BR,2023-02-27,2023-02-27 06:30:01,2023-02-27 17:35:54
175,VSF,2023-02-27 04:54:00,0.2500,-SN BR,2023-02-27,2023-02-27 06:30:01,2023-02-27 17:35:54
176,VSF,2023-02-27 05:54:00,0.2500,-SN BR,2023-02-27,2023-02-27 06:30:01,2023-02-27 17:35:54
177,VSF,2023-02-27 06:54:00,0.2500,-SN BR,2023-02-27,2023-02-27 06:30:01,2023-02-27 17:35:54


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
12,VSF,2020-12-17 07:54:00,3.0500,+SN FZFG,2020-12-17,2020-12-17 07:17:16,2020-12-17 16:15:24
13,VSF,2020-12-17 08:54:00,4.0600,+SN FZFG,2020-12-17,2020-12-17 07:17:16,2020-12-17 16:15:24
14,VSF,2020-12-17 09:54:00,3.5600,+SN FZFG,2020-12-17,2020-12-17 07:17:16,2020-12-17 16:15:24
19,VSF,2020-12-28 15:54:00,0.0001,-SN,2020-12-28,2020-12-28 07:22:02,2020-12-28 16:21:38
23,VSF,2021-01-02 13:54:00,2.0300,SN FG,2021-01-02,2021-01-02 07:22:37,2021-01-02 16:25:52
...,...,...,...,...,...,...,...
157,VSF,2021-02-27 10:54:00,0.0001,-SN,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31
158,VSF,2021-02-27 11:54:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31
159,VSF,2021-02-27 12:54:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31
160,VSF,2021-02-27 13:54:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
13,VSF,2020-12-17 08:54:00,4.0600,+SN FZFG,2020-12-17,2020-12-17 07:17:16,2020-12-17 16:15:24,2020-12-17 08:17:16,2020-12-17 15:15:24
14,VSF,2020-12-17 09:54:00,3.5600,+SN FZFG,2020-12-17,2020-12-17 07:17:16,2020-12-17 16:15:24,2020-12-17 08:17:16,2020-12-17 15:15:24
23,VSF,2021-01-02 13:54:00,2.0300,SN FG,2021-01-02,2021-01-02 07:22:37,2021-01-02 16:25:52,2021-01-02 08:22:37,2021-01-02 15:25:52
24,VSF,2021-01-02 14:54:00,1.0200,-SN BR,2021-01-02,2021-01-02 07:22:37,2021-01-02 16:25:52,2021-01-02 08:22:37,2021-01-02 15:25:52
33,VSF,2021-01-16 08:54:00,1.7800,-SN BR,2021-01-16,2021-01-16 07:18:52,2021-01-16 16:41:08,2021-01-16 08:18:52,2021-01-16 15:41:08
...,...,...,...,...,...,...,...,...,...
157,VSF,2021-02-27 10:54:00,0.0001,-SN,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31,2021-02-27 07:29:14,2021-02-27 16:36:31
158,VSF,2021-02-27 11:54:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31,2021-02-27 07:29:14,2021-02-27 16:36:31
159,VSF,2021-02-27 12:54:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31,2021-02-27 07:29:14,2021-02-27 16:36:31
160,VSF,2021-02-27 13:54:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:29:14,2021-02-27 17:36:31,2021-02-27 07:29:14,2021-02-27 16:36:31


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.7151163934426229
2021-2022: 0.3990387755102041
2022-2023: 0.6627477272727272


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,VSF,2020-12-02 20:54:00,0.0001,-SN,2020-12-02,2020-12-02 07:04:16,2020-12-02 16:14:40
1,VSF,2020-12-03 01:54:00,0.0001,-SN,2020-12-03,2020-12-03 07:05:19,2020-12-03 16:14:25
2,VSF,2020-12-05 22:54:00,0.2500,-SN,2020-12-05,2020-12-05 07:07:21,2020-12-05 16:14:04
3,VSF,2020-12-05 23:54:00,0.2500,-SN,2020-12-05,2020-12-05 07:07:21,2020-12-05 16:14:04
4,VSF,2020-12-06 00:54:00,0.0001,-SN,2020-12-06,2020-12-06 07:08:20,2020-12-06 16:13:57
...,...,...,...,...,...,...,...
151,VSF,2021-02-22 19:54:00,1.5200,SN FZFG,2021-02-22,2021-02-22 06:37:14,2021-02-22 17:30:06
152,VSF,2021-02-22 20:54:00,1.2700,SN FZFG,2021-02-22,2021-02-22 06:37:14,2021-02-22 17:30:06
153,VSF,2021-02-22 21:54:00,1.5200,SN FZFG,2021-02-22,2021-02-22 06:37:14,2021-02-22 17:30:06
154,VSF,2021-02-22 22:54:00,1.5200,-SN BR,2021-02-22,2021-02-22 06:37:14,2021-02-22 17:30:06


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_31920\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.304825
2021-2022: 0.12893962264150943
2022-2023: 0.6113296296296298
